In [1]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt
import warnings
warnings.filterwarnings('ignore')

In [2]:
start = '2016-01-01 UTC'
end = '2020-01-01 UTC'
prices = vbt.YFData.download(
    'AAPL',
    start= start,
    end= end
).get('Close')

In [3]:
prices.vbt.rolling_split(
    n= 30,
    window_len= 365 * 2,
    set_lens= (180,),
    left_to_right= False,
    plot= True
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#1f77b4'], [1.0, '#1f77b4']],
              'hoverongaps': False,
              'name': '0',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': 'ec007eb0-a814-49ed-9d6a-b52aa93818fb',
              'x': array([datetime.datetime(2016, 1, 4, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 5, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 6, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2019, 12, 27, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 30, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 31, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15

In [25]:
(in_price, in_indexes), (out_price, out_indexes) = prices.vbt.rolling_split(
    n= 30,
    window_len= 365 * 2,
    set_lens= (180,),
    left_to_right= False,
)

In [29]:
def simulate_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(
        price,
        windows,
        r=2,
        short_names=['fast', 'slow']
    )

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    fast_ma = vbt.MA.run(
        price,
        window= best_fast_windows,
        per_column= True
    )
    slow_ma = vbt.MA.run(
        price,
        window= best_slow_windows,
        per_column= True
    )

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()


In [39]:
def get_best_index(performance):
    return performance[
        performance.groupby('split_idx').idxmax()
    ].index

def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [31]:
in_sharpe = simulate_all_params(
    in_price,
    np.arange(10, 40),
    direction= 'both',
    freq= 'd'
)

In [35]:
in_sharpe.vbt.plot()

FigureWidget({
    'data': [{'name': 'sharpe_ratio',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0aab5a09-c3b9-4da8-af15-b09962ba5c59',
              'x': [(10, 11, 0), (10, 11, 1), (10, 11, 2), ..., (38, 39, 27), (38,
                    39, 28), (38, 39, 29)],
              'y': array([ 1.48299287,  1.31566883,  1.3825954 , ..., -0.87532807, -0.8006507 ,
                          -0.69530748])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [41]:
in_best_index = get_best_index(in_sharpe)

in_best_fast_windows = get_best_params(
    in_best_index,
    'fast_window'
)

in_best_slow_windows = get_best_params(
    in_best_index,
    'slow_window'
)

in_best_window_pairs = np.array(
    list(
        zip(
            in_best_fast_windows,
            in_best_slow_windows
        )
    )
)

out_test_sharpe = simulate_best_params(
    out_price,
    in_best_fast_windows,
    in_best_slow_windows,
    direction= 'both',
    freq= 'd'
)

In [45]:
t, p = stats.ttest_ind(
    a= out_test_sharpe.values,
    b= in_sharpe[in_best_index].values,
    alternative= 'greater'
)
print(f't-value = {t}') 
print(f'P-value = {p}') # P should be < 0.05 to be statistically significant

t-value = -1.084920983976103
P-value = 0.8587771456189182
